# Retention Prediction with Tensorflow
Ref : https://www.digitalocean.com/community/tutorials/how-to-build-a-deep-learning-model-to-predict-employee-retention-using-keras-and-tensorflow

# Load Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
df = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")

In [2]:
df.rename(columns={"Work_accident": "work_accident", "salary": "salary_level"}, inplace=True)
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,work_accident,left,promotion_last_5years,department,salary_level
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
feats = ['department','salary_level']
df_final = pd.get_dummies(df,columns=feats,drop_first=False)

In [4]:
print(df["department"].unique())
print(df["department"].nunique())

['sales' 'accounting' 'hr' 'technical' 'support' 'management' 'IT'
 'product_mng' 'marketing' 'RandD']
10


In [5]:
df_final.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'work_accident', 'left',
       'promotion_last_5years', 'department_IT', 'department_RandD',
       'department_accounting', 'department_hr', 'department_management',
       'department_marketing', 'department_product_mng', 'department_sales',
       'department_support', 'department_technical', 'salary_level_high',
       'salary_level_low', 'salary_level_medium'],
      dtype='object')

## Separating Train and Test Dataset

In [6]:
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Train Model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

def build_model():
    classifier = Sequential()
    classifier.add(Dense(10, kernel_initializer = "uniform", activation = "relu", input_dim=20))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(
        optimizer = 'adam',
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )
    return classifier

2024-01-02 14:55:56.504056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
params = {
    'batch_size':[10,20,30,40,50],
    'epochs':[2,4,6,8,10,12,14],
    'optimizer':['adam']
}

In [10]:
classifier = KerasClassifier(
    model=build_model,
    optimizer=params["optimizer"],
    verbose=2
)

In [11]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy")

In [12]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
840/840 - 1s - loss: 0.4490 - accuracy: 0.8090 - 1s/epoch - 2ms/step
Epoch 2/2
840/840 - 1s - loss: 0.2787 - accuracy: 0.8897 - 942ms/epoch - 1ms/step
210/210 - 0s - 253ms/epoch - 1ms/step
Epoch 1/2
840/840 - 1s - loss: 0.4555 - accuracy: 0.7825 - 1s/epoch - 2ms/step
Epoch 2/2
840/840 - 1s - loss: 0.3145 - accuracy: 0.8205 - 910ms/epoch - 1ms/step
210/210 - 0s - 167ms/epoch - 796us/step
Epoch 1/2
840/840 - 1s - loss: 0.4508 - accuracy: 0.8019 - 1s/epoch - 1ms/step
Epoch 2/2
840/840 - 1s - loss: 0.2773 - accuracy: 0.8947 - 943ms/epoch - 1ms/step
210/210 - 0s - 192ms/epoch - 912us/step
Epoch 1/2
840/840 - 1s - loss: 0.4610 - accuracy: 0.7624 - 1s/epoch - 2ms/step
Epoch 2/2
840/840 - 1s - loss: 0.3394 - accuracy: 0.8051 - 881ms/epoch - 1ms/step
210/210 - 0s - 207ms/epoch - 986us/step
Epoch 1/2
840/840 - 1s - loss: 0.4394 - accuracy: 0.8107 - 1s/epoch - 1ms/step
Epoch 2/2
840/840 - 1s - loss: 0.2645 - accuracy: 0.8994 - 822ms/epoch - 978us/step
210/210 - 0s - 172ms/epoch - 818us/

In [13]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_
print(best_param)
print(best_accuracy)

{'batch_size': 10, 'epochs': 8, 'optimizer': 'adam'}
0.9520911998911046


In [14]:
best_classifier = KerasClassifier(
    model = build_model,
    batch_size=best_param.get('batch_size'),
    epochs=best_param.get('epochs'),
    optimizer=best_param.get('optimizer')
    )

In [15]:
accuracies = cross_val_score(
    estimator = best_classifier,
    X = X_train,
    y = y_train,
    cv = 10,
    n_jobs = -1
)

Epoch 1/8
Epoch 1/8
Epoch 1/8
Epoch 1/8
Epoch 1/8
Epoch 1/8
Epoch 1/8
Epoch 1/8
945/945 [==============================] - 10s 7ms/step - loss: 0.4282 - accuracy: 0.8202
Epoch 2/8
945/945 [==============================] - 10s 7ms/step - loss: 0.4611 - accuracy: 0.7599
Epoch 2/8
945/945 [==============================] - 10s 7ms/step - loss: 0.4516 - accuracy: 0.7625
Epoch 2/8
945/945 [==============================] - 6s 6ms/step - loss: 0.2480 - accuracy: 0.9137
Epoch 3/8
945/945 [==============================] - 6s 6ms/step - loss: 0.2576 - accuracy: 0.9051
Epoch 3/8
945/945 [==============================] - 6s 6ms/step - loss: 0.3358 - accuracy: 0.8253
Epoch 3/8
945/945 [==============================] - 6s 6ms/step - loss: 0.2599 - accuracy: 0.8590
Epoch 4/8
945/945 [==============================] - 6s 6ms/step - loss: 0.3026 - accuracy: 0.8844
Epoch 4/8
945/945 [==============================] - 6s 6ms/step - loss: 0.2147 - accuracy: 0.9260
Epoch 4/8
945/945 [=================

In [16]:
mean = accuracies.mean()
variance = accuracies.var()
print(f"accuracy mean: {mean}; accuracy variance: {variance}")

accuracy mean: 0.9464711062690091; accuracy variance: 0.00010517743412362838


In [17]:
best_classifier.fit(
    X_train,
    y_train,
    batch_size = best_param.get('batch_size'),
    epochs = best_param.get('epochs')
)

Epoch 1/8
1050/1050 [==============================] - 3s 2ms/step - loss: 0.4513 - accuracy: 0.7617
Epoch 2/8
1050/1050 [==============================] - 2s 2ms/step - loss: 0.3289 - accuracy: 0.8364
Epoch 3/8
1050/1050 [==============================] - 2s 1ms/step - loss: 0.2896 - accuracy: 0.8990
Epoch 4/8
1050/1050 [==============================] - 2s 1ms/step - loss: 0.2703 - accuracy: 0.9128
Epoch 5/8
1050/1050 [==============================] - 2s 2ms/step - loss: 0.2547 - accuracy: 0.9174
Epoch 6/8
1050/1050 [==============================] - 2s 2ms/step - loss: 0.2477 - accuracy: 0.9174
Epoch 7/8
1050/1050 [==============================] - 1s 1ms/step - loss: 0.2364 - accuracy: 0.9240
Epoch 8/8
1050/1050 [==============================] - 2s 1ms/step - loss: 0.2356 - accuracy: 0.9249


KerasClassifier(
	model=<function build_model at 0x1482568e0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=10
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=8
	class_weight=None
)

In [18]:
y_pred = (best_classifier.predict(X_test) > 0.5).astype("bool")
y_pred

450/450 [==============================] - 1s 1ms/step


array([False, False, False, ..., False, False, False])

# Confusion Matrix

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3272,  151],
       [ 158,  919]])

In [20]:
new_pred = best_classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.,0.]])))
new_pred = (new_pred > 0.5)
print(new_pred)

1/1 [==============================] - 0s 49ms/step
[False]


# Save and Load model

## Save into HDF5 format

In [21]:
import pickle
import numpy as np
# Create model folder
folder = "../2-flask-api/model/hdf5/v1/"
!mkdir -p ../2-flask-api/model/hdf5/v1/

In [22]:
# saving model
json_model = best_classifier.model_.to_json()
with open(f'{folder}retention_prediction_model.json', 'w') as handler:
    handler.write(json_model)
    
# saving weights
best_classifier.model_.save_weights(f'{folder}retention_prediction_weights.h5', overwrite=True)

# saving standard scaler
with open(f'{folder}standard_scaler.pkl', 'wb') as handler:
    pickle.dump(sc, handler)

In [23]:
# loading model
from tensorflow.keras.models import model_from_json

model = model_from_json(open(f'{folder}retention_prediction_model.json').read())
model.load_weights(f'{folder}retention_prediction_weights.h5')

# dont forget to compile your model
model.compile(loss='binary_crossentropy', optimizer='adam')

# and load standard_scaler
with open(f'{folder}standard_scaler.pkl', 'rb') as handler:
    sc = pickle.load(handler)

In [24]:
model_pred = model.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.,0.]])))
print(model_pred)

1/1 [==============================] - 0s 50ms/step
[[0.12258662]]
